In [1]:
import os
import wget
import argparse
import pm4py
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import metrics 

from processtransformer import constants
from processtransformer.models import transformer
from processtransformer.data.loader import LogsDataLoader
from processtransformer.data.processor import LogsDataProcessor


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [29]:
data_dir = "./datasets/"
_ = wget.download("https://data.4tu.nl/file/94ee26c8-78f6-4387-b32b-f028f2103a2c/697ff9af-fca0-4363-bd84-e81735d27e9f")


,org:group,resource country,organization country,org:resource,organization involved,org:role,concept:name,impact,product,lifecycle:transition,time:timestamp,case:concept:name
0,Org line A2,INDIA,se,Minnie,J11 2nd,A2_2,Queued,High,PROD191,Awaiting Assignment,2006-01-11 14:49:42+00:00,1-109135791
1,Org line A2,INDIA,se,Minnie,J11 2nd,A2_2,Accepted,High,PROD191,In Progress,2012-03-15 10:53:52+00:00,1-109135791
2,Org line A2,INDIA,se,Minnie,J11 2nd,A2_2,Accepted,High,PROD191,Assigned,2012-03-15 10:56:17+00:00,1-109135791
3,Org line A2,INDIA,se,Minnie,J11 2nd,A2_2,Accepted,High,PROD191,In Progress,2012-03-15 11:09:05+00:00,1-109135791
4,Org line A2,INDIA,se,Minnie,J11 2nd,A2_2,Completed,High,PROD191,Closed,2012-03-15 11:11:33+00:00,1-109135791
...,...,...,...,...,...,...,...,...,...,...,...,...
6655,Org line C,Sweden,se,Karl,G161 2nd,E_7,Completed,Medium,PROD671,Closed,2012-05-30 13:04:30+00:00,1-752134249
6656,Org line A2,Sweden,se,Niklas,U6 2nd,A2_3,Accepted,Major,PROD831,In Progress,2012-05-31 09:58:45+00:00,1-752600115
6657,Org line A2,Sweden,se,Niklas,U6 2nd,A2_3,Completed,Major,PROD831,Closed,2012-05-31 10:01:24+00:00,1-752600115
6658,Org line G3,POLAND,us,Ewa,G199 3rd,NaN,Accepted,High,PROD97,In Progress,2012-05-31 18:07:59+00:00,1-752835764


In [10]:
data_processor = LogsDataProcessor(name='Helpdesk', filepath="finale.csv",  
                                    columns = ["Case ID", "Activity", "Complete Timestamp"], #specify the columns name containing case_id, activity name and timestamp 
                                    dir_path='datasets', pool = 4)
data_processor.process_logs(task=constants.Task.NEXT_ACTIVITY, sort_temporally= False)

In [11]:
# Load data
data_loader = LogsDataLoader(name = 'Helpdesk')

(train_df, test_df, x_word_dict, y_word_dict, max_case_length, 
    vocab_size, num_output) = data_loader.load_data(constants.Task.NEXT_ACTIVITY)

# Prepare training examples for next activity prediction task
train_token_x, train_token_y = data_loader.prepare_data_next_activity(train_df, 
    x_word_dict, y_word_dict, max_case_length)

In [12]:
learning_rate = 0.001
batch_size = 12
epochs = 10

In [13]:
# Create and train a transformer model
transformer_model = transformer.get_next_activity_model(
    max_case_length=max_case_length, 
    vocab_size=vocab_size,
    output_dim=num_output)

transformer_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

transformer_model.fit(train_token_x, train_token_y, 
    epochs=epochs, batch_size=batch_size)

Epoch 1/10
1121/1121 [==============================] - 26s 13ms/step - loss: 0.8139 - sparse_categorical_accuracy: 0.7392
Epoch 2/10
1121/1121 [==============================] - 14s 13ms/step - loss: 0.6542 - sparse_categorical_accuracy: 0.7907
Epoch 3/10
1121/1121 [==============================] - 14s 13ms/step - loss: 0.6448 - sparse_categorical_accuracy: 0.7898
Epoch 4/10
1121/1121 [==============================] - 16s 15ms/step - loss: 0.6367 - sparse_categorical_accuracy: 0.7921
Epoch 5/10
1121/1121 [==============================] - 15s 14ms/step - loss: 0.6327 - sparse_categorical_accuracy: 0.7926
Epoch 6/10
1121/1121 [==============================] - 14s 13ms/step - loss: 0.6336 - sparse_categorical_accuracy: 0.7934
Epoch 7/10
1121/1121 [==============================] - 14s 12ms/step - loss: 0.6330 - sparse_categorical_accuracy: 0.7923
Epoch 8/10
1121/1121 [==============================] - 14s 12ms/step - loss: 0.6280 - sparse_categorical_accuracy: 0.7921
Epoch 9/10
1121/

In [14]:
# Evaluate over all the prefixes (k) and save the results
k, accuracies,fscores, precisions, recalls = [],[],[],[],[]
for i in range(max_case_length):
    test_data_subset = test_df[test_df["k"]==i]
    if len(test_data_subset) > 0:
        test_token_x, test_token_y = data_loader.prepare_data_next_activity(test_data_subset, 
            x_word_dict, y_word_dict, max_case_length)   
        y_pred = np.argmax(transformer_model.predict(test_token_x), axis=1)
        accuracy = metrics.accuracy_score(test_token_y, y_pred)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(
            test_token_y, y_pred, average="weighted")
        k.append(i)
        accuracies.append(accuracy)
        fscores.append(fscore)
        precisions.append(precision)
        recalls.append(recall)

k.append(i + 1)
accuracies.append(np.mean(accuracy))
fscores.append(np.mean(fscores))
precisions.append(np.mean(precisions))
recalls.append(np.mean(recalls))

1/1 [==============================] - 0s 84ms/step


In [15]:
print('Average accuracy across all prefixes:', np.mean(accuracies))
print('Average f-score across all prefixes:', np.mean(fscores))
print('Average precision across all prefixes:', np.mean(precisions))
print('Average recall across all prefixes:', np.mean(recalls))

Average accuracy across all prefixes: 0.8465945429046428
Average f-score across all prefixes: 0.7946957040059228
Average precision across all prefixes: 0.8022292362341974
Average recall across all prefixes: 0.833810754813363


In [16]:
data_processor = LogsDataProcessor(name='Helpdesk', filepath="finale.csv",  
                                    columns = ["Case ID", "Activity", "Complete Timestamp"],  #specify the columns name containing case_id, activity name and timestamp 
                                    dir_path='datasets', pool = 4)
data_processor.process_logs(task=constants.Task.NEXT_TIME, sort_temporally= False)

In [17]:
# Load data
data_loader = LogsDataLoader(name = 'Helpdesk')

(train_df, test_df, x_word_dict, y_word_dict, max_case_length, 
    vocab_size, num_output) = data_loader.load_data(constants.Task.NEXT_TIME)

# Prepare training examples for next time prediction task
(train_token_x, train_time_x, train_token_y, time_scaler, y_scaler) = \
                                    data_loader.prepare_data_next_time(train_df, x_word_dict, max_case_length)

In [18]:
learning_rate = 0.001
batch_size = 12
epochs = 1

In [19]:
# Create and train a transformer model
transformer_model = transformer.get_next_time_model(
        max_case_length=max_case_length, 
        vocab_size=vocab_size)

transformer_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
        loss=tf.keras.losses.LogCosh())

transformer_model.fit([train_token_x, train_time_x], train_token_y, 
        epochs=epochs, batch_size=batch_size, verbose=2)

1426/1426 - 28s - loss: 0.1573 - 28s/epoch - 20ms/step


In [20]:
# Evaluate over all the prefixes (k) and save the results
k, maes, mses, rmses = [],[],[],[]
for i in range(max_case_length):
    test_data_subset = test_df[test_df["k"]==i]
    if len(test_data_subset) > 0:
        test_token_x, test_time_x, test_y, _, _ = data_loader.prepare_data_next_time(
            test_data_subset, x_word_dict, max_case_length, time_scaler, y_scaler, False)   

        y_pred = transformer_model.predict([test_token_x, test_time_x])
        _test_y = y_scaler.inverse_transform(test_y)
        _y_pred = y_scaler.inverse_transform(y_pred)

        k.append(i)
        maes.append(metrics.mean_absolute_error(_test_y, _y_pred))
        mses.append(metrics.mean_squared_error(_test_y, _y_pred))
        rmses.append(np.sqrt(metrics.mean_squared_error(_test_y, _y_pred)))

k.append(i + 1)
maes.append(np.mean(maes))
mses.append(np.mean(mses))
rmses.append(np.mean(rmses))  

1/1 [==============================] - 0s 92ms/step


In [21]:
print('Average MAE across all prefixes:', np.mean(maes))
print('Average MSE across all prefixes:', np.mean(mses))
print('Average RMSE across all prefixes:', np.mean(rmses))

Average MAE across all prefixes: 3.538343
Average MSE across all prefixes: 33.242516
Average RMSE across all prefixes: 5.3476806


In [22]:
data_processor = LogsDataProcessor(name='Helpdesk', filepath="finale.csv",  
                                    columns = ["Case ID", "Activity", "Complete Timestamp"],  #specify the columns name containing case_id, activity name and timestamp 
                                    dir_path='datasets', pool = 4)
data_processor.process_logs(task=constants.Task.REMAINING_TIME, sort_temporally= False)

In [23]:
# Load data
data_loader = LogsDataLoader(name = 'Helpdesk')

(train_df, test_df, x_word_dict, y_word_dict, max_case_length, 
    vocab_size, num_output) = data_loader.load_data(constants.Task.REMAINING_TIME)

# Prepare training examples for next time prediction task
(train_token_x, train_time_x, 
    train_token_y, time_scaler, y_scaler) = data_loader.prepare_data_remaining_time(train_df, 
    x_word_dict, max_case_length)

In [24]:
learning_rate = 0.001
batch_size = 12
epochs = 1

In [26]:
# Create and train a transformer model
transformer_model = transformer.get_remaining_time_model(
    max_case_length=max_case_length, 
    vocab_size=vocab_size)

transformer_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
    loss=tf.keras.losses.LogCosh())

transformer_model.fit([train_token_x, train_time_x], train_token_y, 
        epochs=epochs, batch_size=batch_size, verbose=2)

1426/1426 - 33s - loss: 0.0919 - 33s/epoch - 23ms/step


In [27]:
# Evaluate over all the prefixes (k) and save the results
k, maes, mses, rmses = [],[],[],[]
for i in range(max_case_length):
    test_data_subset = test_df[test_df["k"]==i]
    if len(test_data_subset) > 0:
        test_token_x, test_time_x, test_y, _, _ = data_loader.prepare_data_remaining_time(
            test_data_subset, x_word_dict, max_case_length, time_scaler, y_scaler, False)   

        y_pred = transformer_model.predict([test_token_x, test_time_x])
        _test_y = y_scaler.inverse_transform(test_y)
        _y_pred = y_scaler.inverse_transform(y_pred)

        k.append(i)
        maes.append(metrics.mean_absolute_error(_test_y, _y_pred))
        mses.append(metrics.mean_squared_error(_test_y, _y_pred))
        rmses.append(np.sqrt(metrics.mean_squared_error(_test_y, _y_pred)))

k.append(i + 1)
maes.append(np.mean(maes))
mses.append(np.mean(mses))
rmses.append(np.mean(rmses))  

1/1 [==============================] - 0s 80ms/step


In [28]:
print('Average MAE across all prefixes:', np.mean(maes))
print('Average MSE across all prefixes:', np.mean(mses))
print('Average RMSE across all prefixes:', np.mean(rmses))

Average MAE across all prefixes: 3.7265153
Average MSE across all prefixes: 30.006725
Average RMSE across all prefixes: 4.9517913
